In [ ]:
from collections import Counter
from textblob import TextBlob
import tweepy
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
# Replace with your own Twitter API credentials
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# List of hashtags
hashtags = ["", "", ""]

# List to store the tweets and set to store the ids
all_tweets = []
tweet_ids = set()

# Get the list of stopwords in English
stop_words = set(stopwords.words('english'))

for hashtag in hashtags:
    tweets = tweepy.Cursor(api.search, q=hashtag, lang='en', tweet_mode='extended').items(1000)
    for tweet in tweets:
        if tweet.id not in tweet_ids:
            all_tweets.append(tweet)
            tweet_ids.add(tweet.id)

positive_tweets = []
negative_tweets = []

for tweet in all_tweets:
    tweet_text = tweet.full_text
    # Preprocessing
    tweet_text = re.sub(r"http\S+|www\S+|https\S+", '', tweet_text, flags=re.MULTILINE)
    tweet_text = re.sub(r'\@\w+|\#','', tweet_text)
    tweet_text = tweet_text.lower()

    # Tokenize the tweet text and remove stopwords
    word_tokens = word_tokenize(tweet_text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = ' '.join(filtered_sentence)

    # Sentiment analysis
    blob = TextBlob(filtered_sentence)
    sentiment = blob.sentiment.polarity
    if sentiment > 0:
        positive_tweets.append(filtered_sentence)
    elif sentiment < 0:
        negative_tweets.append(filtered_sentence)

# Getting most common words in positive tweets
positive_counter = Counter(" ".join(positive_tweets).split())
common_positive_words = positive_counter.most_common(10)

# Getting most common words in negative tweets
negative_counter = Counter(" ".join(negative_tweets).split())
common_negative_words = negative_counter.most_common(10)

print("Common words in positive tweets:")
for word, count in common_positive_words:
    print(word, count)

print("\nCommon words in negative tweets:")
for word, count in common_negative_words:
    print(word, count)